In [ ]:
# default_exp type_emb
# all_data

In [ ]:
# hide
%load_ext autoreload
%autoreload 2

# Type embeddings

## Imports

In [ ]:
from gensim.models import Word2Vec

## Pre-processing

In [ ]:
from socemb.read_data import *

In [ ]:
fpaths = get_fpaths('Anglo-Saxon', source='local')

In [ ]:
df = read_comments(fpaths)

In [ ]:
docs = df['body']

In [ ]:
docs = docs.dropna()

In [ ]:
docs = docs.str.lower()

In [ ]:
docs = docs.str.split()

In [ ]:
docs[0]

['current',
 'playthrough....',
 'no',
 'cheat',
 'ironman',
 'playing',
 'three',
 'duchy',
 'king',
 '(iceland,',
 'orkney',
 '&amp;',
 'vestlandet).',
 'i',
 'converted',
 'to',
 'christianity',
 'pretty',
 'early',
 '(an',
 'early',
 'holy',
 'war',
 'that',
 'turned',
 'saxony',
 'into',
 'christians',
 'really',
 'saved',
 'me',
 'here)',
 'and',
 'was',
 'able',
 'to',
 'stay',
 'holy',
 'wars',
 'with',
 'all',
 'my',
 'neighbors.',
 'which',
 'was',
 'good',
 'that',
 'i',
 'kept',
 'a',
 'raised',
 'raiding',
 'army',
 'around',
 'for',
 'a',
 'while',
 'after',
 'to',
 'converted',
 'to',
 'feudalism.',
 'i',
 'turned',
 'the',
 'duchy',
 'of',
 'iceland',
 'into',
 'a',
 'merchant',
 'republic',
 'and',
 'that',
 'relationship',
 'has',
 'been',
 'spotty',
 'at',
 'best.',
 'but',
 'the',
 'real',
 'fun',
 'is',
 'fucking',
 'with',
 'my',
 'neighbors.',
 'i',
 'had',
 'a',
 'character',
 'with',
 'really',
 'high',
 'intrigue',
 'and',
 'diplomacy',
 'but',
 'shit',
 'for'

In [ ]:
len(docs)

138928

In [ ]:
sents =[
    ['I', 'like', 'playing', 'basketball', 'and', 'tennis'],
    ['You', 'like', 'reading', 'and', 'writing']
]

In [ ]:
# export
class Corpus:
    """An iterator that yields sentences (lists of str)."""
    def __init__(self, docs):
        self.docs = docs
    def __iter__(self):
        for doc in docs:
            yield doc

In [ ]:
corpus = Corpus(docs)

## Train embeddings

In [ ]:
%%time
model = Word2Vec(
    corpus, 
    min_count=10,
    size=100,
    workers=3, 
    window=3
)

In [ ]:
model.wv['basketball'].shape

(50,)

## Evaluate embeddings

In [ ]:
for index, word in enumerate(model.wv.index2word):
    if index == 10:
        break
    print(f"word #{index}/{len(model.wv.index2word)} is {word}")

word #0/625086 is the
word #1/625086 is of
word #2/625086 is and
word #3/625086 is to
word #4/625086 is a
word #5/625086 is in
word #6/625086 is is
word #7/625086 is that
word #8/625086 is i
word #9/625086 is as


In [ ]:
model.wv.similarity('basketball', 'tennis')

0.87182456

In [ ]:
model.wv.similarity('basketball', 'i')

0.5944799

In [ ]:
model.wv.most_similar('basketball')[:10]

[('baseball', 0.9226516485214233),
 ('hockey', 0.8914705514907837),
 ('soccer', 0.890144407749176),
 ('chess', 0.8887145519256592),
 ('golf', 0.8880098462104797),
 ('cricket', 0.8862640261650085),
 ('billionaire', 0.8857084512710571),
 ('manager', 0.8841858506202698),
 ('baltimore', 0.8819049596786499),
 ('team', 0.8810849189758301)]